In [1]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import confusion_matrix
from PIL import Image

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.6.0


In [2]:
%config Completer.use_jedi = False
print('ok')

ok


In [3]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [4]:
# Dataset folders 
# dataset_in = 'training'
dataset_out = 'output'
training_dir = '../input/dataset/content/output/train'
validation_dir = '../input/dataset/content/output/val'


In [5]:
#split.ratio(dataset_in, output=dataset_out, seed=seed, ratio=(.8,.1,.1), group_prefix=None)

In [6]:
labels = {0 : 'Apple',       # 0
          1 : 'Blueberry',           # 1
          2 : 'Cherry',    # 2
          3 : 'Corn',              # 3
          4 : 'Grape',          # 4
          5 : 'Orange',          # 5
          6 : 'Peach',   # 6
          7 : 'Pepper',             # 7
          8 : 'Potato',            # 8
          9 : 'Raspberry',         # 9
          10 : 'Soybean',             # 10
          11 : 'Squash',       # 11
          12 : 'Strawberry',  # 12
          13 : 'Tomato'   # 13
         }

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of ImageDataGenerator for training, validation sets

# Create an instance of ImageDataGenerator for training, validation, and test sets
train_data_gen = ImageDataGenerator()
valid_data_gen = ImageDataGenerator()

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method
train_gen = train_data_gen.flow_from_directory(directory=training_dir,
                                               target_size=(256,256),
                                               interpolation='lanczos',
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=32,
                                               shuffle=True,
                                               seed=seed)
valid_gen = train_data_gen.flow_from_directory(directory=validation_dir,
                                               target_size=(256,256),
                                               interpolation='lanczos',
                                               color_mode='rgb',
                                               classes=None, # can be set to labels
                                               class_mode='categorical',
                                               batch_size=32,
                                               shuffle=False,
                                               seed=seed)

Found 14176 images belonging to 14 classes.
Found 3552 images belonging to 14 classes.


In [8]:
input_shape = (256, 256, 3)
epochs = 100

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

img_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.4),
        layers.RandomTranslation(height_factor=0.4, width_factor=0.4),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.3),
    ],
    name="img_augmentation",
)

2021-11-19 19:35:55.145098: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-19 19:35:55.245678: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-19 19:35:55.246389: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-19 19:35:55.248775: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [10]:
from tensorflow.keras.applications import * #Efficient Net included here
from tensorflow.keras import models
from tensorflow.keras import layers
from keras.preprocessing.image import ImageDataGenerator
import os
import shutil
import pandas as pd
from sklearn import model_selection
from tqdm import tqdm
from tensorflow.keras import optimizers
import tensorflow as tf

def build_efficientNetB7(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = img_augmentation(inputs)
    model = EfficientNetB7(include_top=False, input_tensor=x, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.4
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(14, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [11]:
def unfreeze_model(model):
    # We unfreeze the top 20 layers while leaving BatchNorm layers frozen
    for layer in model.layers[-30:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )

In [12]:
model = build_efficientNetB7(input_shape)
unfreeze_model(model)

#model.summary()

258088960/258076736 [==============================] - 4s 0us/step


In [13]:
# Utility function to create folders and callbacks for training
from datetime import datetime

def create_folders_and_callbacks(model_name):

  exps_dir = os.path.join('data_augmentation_experiments')
  if not os.path.exists(exps_dir):
      os.makedirs(exps_dir)

  now = datetime.now().strftime('%b%d_%H-%M-%S')

  exp_dir = os.path.join(exps_dir, model_name + '_' + str(now))
  if not os.path.exists(exp_dir):
      os.makedirs(exp_dir)
      
  callbacks = []

  # Model checkpoint
  # ----------------
  ckpt_dir = os.path.join(exp_dir, 'ckpts')
  if not os.path.exists(ckpt_dir):
      os.makedirs(ckpt_dir)

  ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp.ckpt'), 
                                                     save_weights_only=False, # True to save only weights
                                                     save_best_only=True) # True to save only the best epoch 
  callbacks.append(ckpt_callback)

  # Visualize Learning on Tensorboard
  # ---------------------------------
  tb_dir = os.path.join(exp_dir, 'tb_logs')
  if not os.path.exists(tb_dir):
      os.makedirs(tb_dir)
      
  # By default shows losses and metrics for both training and validation
  tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir, 
                                               profile_batch=0,
                                               histogram_freq=1)  # if > 0 (epochs) shows weights histograms
  callbacks.append(tb_callback)

  # Early Stopping
  # --------------
  es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
  callbacks.append(es_callback)

  return callbacks

In [14]:
# Create folders and callbacks and fit
aug_callbacks = create_folders_and_callbacks(model_name='CNN_Aug')
# Train the model
history = model.fit(
    x = train_gen,
    epochs = epochs,
    validation_data = valid_gen,
    callbacks = aug_callbacks,
).history

/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)
2021-11-19 19:36:08.182284: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


2021-11-19 19:36:26.415671: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


443/443 [==============================] - 198s 394ms/step - loss: 0.7265 - accuracy: 0.7903 - val_loss: 0.2577 - val_accuracy: 0.9240


2021-11-19 19:40:02.321856: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


...

Epoch 61/100
443/443 [==============================] - 169s 382ms/step - loss: 0.0535 - accuracy: 0.9824 - val_loss: 0.0203 - val_accuracy: 0.9941
Epoch 62/100
443/443 [==============================] - 169s 383ms/step - loss: 0.0557 - accuracy: 0.9813 - val_loss: 0.0237 - val_accuracy: 0.9930
Epoch 63/100
443/443 [==============================] - 169s 382ms/step - loss: 0.0495 - accuracy: 0.9832 - val_loss: 0.0195 - val_accuracy: 0.9935
Epoch 64/100
443/443 [==============================] - 169s 381ms/step - loss: 0.0515 - accuracy: 0.9846 - val_loss: 0.0183 - val_accuracy: 0.9955
Epoch 65/100
443/443 [==============================] - 169s 381ms/step - loss: 0.0529 - accuracy: 0.9825 - val_loss: 0.0125 - val_accuracy: 0.9958


/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 66/100
443/443 [==============================] - 169s 381ms/step - loss: 0.0522 - accuracy: 0.9833 - val_loss: 0.0148 - val_accuracy: 0.9952
Epoch 67/100
443/443 [==============================] - 169s 381ms/step - loss: 0.0479 - accuracy: 0.9851 - val_loss: 0.0139 - val_accuracy: 0.9952
Epoch 68/100
443/443 [==============================] - 169s 381ms/step - loss: 0.0466 - accuracy: 0.9857 - val_loss: 0.0157 - val_accuracy: 0.9961
Epoch 69/100
443/443 [==============================] - 169s 381ms/step - loss: 0.0427 - accuracy: 0.9855 - val_loss: 0.0171 - val_accuracy: 0.9941
Epoch 70/100
443/443 [==============================] - 169s 381ms/step - loss: 0.0457 - accuracy: 0.9845 - val_loss: 0.0227 - val_accuracy: 0.9938
Epoch 71/100
443/443 [==============================] - 169s 381ms/step - loss: 0.0502 - accuracy: 0.9831 - val_loss: 0.0179 - val_accuracy: 0.9952
Epoch 72/100
443/443 [==============================] - 169s 381ms/step - loss: 0.0444 - accuracy: 0.9850 - val_

/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 75/100
443/443 [==============================] - 169s 381ms/step - loss: 0.0458 - accuracy: 0.9846 - val_loss: 0.0189 - val_accuracy: 0.9938
Epoch 76/100
443/443 [==============================] - 169s 382ms/step - loss: 0.0449 - accuracy: 0.9851 - val_loss: 0.0156 - val_accuracy: 0.9958
Epoch 77/100
443/443 [==============================] - 169s 381ms/step - loss: 0.0424 - accuracy: 0.9867 - val_loss: 0.0183 - val_accuracy: 0.9935
Epoch 78/100
443/443 [==============================] - 169s 382ms/step - loss: 0.0428 - accuracy: 0.9857 - val_loss: 0.0143 - val_accuracy: 0.9958
Epoch 79/100
443/443 [==============================] - 169s 382ms/step - loss: 0.0459 - accuracy: 0.9848 - val_loss: 0.0148 - val_accuracy: 0.9955
Epoch 80/100
443/443 [==============================] - 169s 381ms/step - loss: 0.0410 - accuracy: 0.9870 - val_loss: 0.0188 - val_accuracy: 0.9944
Epoch 81/100
443/443 [==============================] - 169s 381ms/step - loss: 0.0361 - accuracy: 0.9874 - val_